# Attention 메커니즘의 핵심 아이디어

1. 동적 집중(Dynamic Focus)

In [1]:
import numpy as np

# Query, Key, Value 벡터 정의
query = np.array([[1, 0, 1]])
key = np.array([[1, 0, 0], [0, 1, 0], [1, 1, 0]])
value = np.array([[10, 0], [0, 10], [5, 5]])

# 어텐션 점수 계산
scores = np.dot(query, key.T)

# Softmax를 통한 정규화
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

attention_weights = softmax(scores)

# 어텐션 결과 계산
attention_output = np.dot(attention_weights, value)

print("Attention Weights:", attention_weights)
print("Dynamic Focus Output:", attention_output)


Attention Weights: [[0.4223188 0.1553624 0.4223188]]
Dynamic Focus Output: [[6.33478197 3.66521803]]


# Seq2Seq with RNN and Attention (2015) Python 예시

Seq2Seq 모델에 어텐션 메커니즘을 추가한 구조는 2015년 Bahdanau 어텐션으로 잘 알려져 있습니다. 

이 모델은 기계 번역, 텍스트 생성 등 자연어 처리(NLP)에서 널리 사용됩니다. 

코드 설명

In [ ]:
# Attention 메커니즘:
# Attention 클래스는 인코더 출력과 디코더의 현재 은닉 상태를 사용해 어텐션 가중치를 계산합니다.
# Softmax를 통해 가중치를 정규화하고, 중요한 정보에 더 높은 가중치를 부여합니다.

# EncoderRNN:
# 입력 시퀀스를 임베딩하고, GRU를 통해 인코딩합니다.
# 출력은 인코더의 모든 시퀀스 출력과 마지막 은닉 상태입니다.

# DecoderRNN:
# 현재 입력과 인코더의 출력, 이전 은닉 상태를 사용해 예측합니다.
# 어텐션 메커니즘을 사용하여 인코더의 출력 중 중요한 부분에 집중합니다.

# Seq2Seq 모델:
# 인코더와 디코더를 결합하여 전체 Seq2Seq 모델을 구성합니다.
# 입력 시퀀스를 인코딩한 후, 디코더가 반복적으로 예측을 수행합니다.

#         학습 과정:
# 임의의 데이터를 생성하고, 모델을 학습시킵니다.
# 각 에포크마다 손실 값을 출력합니다.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

# 디바이스 설정 (GPU 또는 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 어텐션 메커니즘 정의
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        # hidden 차원 조정
        hidden = hidden[-1].unsqueeze(1)  # (batch_size, 1, hidden_size)
        seq_len = encoder_outputs.size(1)

        # hidden을 seq_len만큼 반복
        hidden = hidden.repeat(1, seq_len, 1)

        # 어텐션 가중치 계산
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = torch.softmax(self.v(energy).squeeze(2), dim=1)

        return attention

# 인코더 정의
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

    def forward(self, input):
        embedded = self.embedding(input)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

# 디코더 정의
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.attention = Attention(hidden_size)

    def forward(self, input, hidden, encoder_outputs):
        # 입력 임베딩
        embedded = self.embedding(input).unsqueeze(1)

        # 어텐션 가중치 계산
        attn_weights = self.attention(hidden, encoder_outputs).unsqueeze(1)

        # 컨텍스트 벡터 계산
        context = torch.bmm(attn_weights, encoder_outputs)

        # GRU 입력 준비
        rnn_input = torch.cat((embedded, context), dim=2)

        # GRU 계산
        output, hidden = self.gru(rnn_input, hidden)

        # 출력 계산
        output = self.fc(torch.cat((output.squeeze(1), context.squeeze(1)), dim=1))

        return output, hidden, attn_weights

# Seq2Seq 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        encoder_outputs, hidden = self.encoder(src)
        outputs = torch.zeros(trg.size(0), trg.size(1), self.decoder.output_size).to(device)

        input = trg[:, 0]
        for t in range(1, trg.size(1)):
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output
            input = output.argmax(dim=1)

        return outputs

# 하이퍼파라미터 설정
input_size = 100
output_size = 100
hidden_size = 256

# 모델 생성 및 디바이스 이동
encoder = EncoderRNN(input_size, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_size).to(device)
model = Seq2Seq(encoder, decoder).to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 임의의 입력 데이터 생성
src = torch.randint(0, input_size, (32, 10)).to(device)
trg = torch.randint(0, output_size, (32, 10)).to(device)

# 학습 예시
model.train()
for epoch in range(1, 6):
    optimizer.zero_grad()
    output = model(src, trg)
    loss = criterion(output.view(-1, output_size), trg.view(-1))
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 4.6105
Epoch 2, Loss: 4.5100
Epoch 3, Loss: 4.4259
Epoch 4, Loss: 4.3364
Epoch 5, Loss: 4.2388


In [ ]:
# 손실 값의 의미
# 크로스 엔트로피 손실(CrossEntropy Loss)
# 이 모델에서는 크로스 엔트로피 손실 함수를 사용했습니다.
# 크로스 엔트로피 손실은 모델이 예측한 확률 분포와 실제 레이블의 차이를 측정합니다.
# 손실 값이 낮을수록 모델의 예측이 실제 값에 더 가까움을 의미합니다.
# b. 손실 값의 감소
# 에포크가 진행됨에 따라 손실 값이 점차 감소하고 있습니다:
# Epoch 1, Loss: 4.6052에서 Epoch 5, Loss: 4.1920로 감소.
# 손실 값이 감소하는 것은 모델이 점점 학습되고 있으며, 입력과 출력 간의 관계를 더 잘 이해하고 있음을 나타냅니다.
# 초기 손실 값이 4.6 근처인 것은 출력이 100개의 클래스 중에서 무작위로 예측할 때의 손실 값과 비슷합니다. 이는 초기 예측이 대부분 랜덤임을 의미합니다.